<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/123_%E6%A7%8B%E9%80%A0%E7%9A%84%E9%83%A8%E5%88%86%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

構造的部分型
============

リスコフの置換原則
------------------

**リスコフの置換原則**（Liskov Substitution Principle; LSP）とは、「プログラムで使用する型 S のオブジェクトを型 T のオブジェクトに置き換えても動作は変わらないならば、S は T の部分型である」というものである。ここで、**部分型**（subtyping）であるとは、ある型に対して、型安全に代入可能とされることを意味している。部分型の代入先となる型を**基底型**（supertype）と呼ぶ。

現在、部分型には次の 2 種類が存在する。

  * **名前的部分型**（nominal subtyping）:  
継承を使って部分型であると宣言された型。C++ や Java で採用されている部分型。
  * **構造的部分型**（structural subtyping）:  
相手が持っている属性とその型をすべて含んでいる型。TypeScript や Go で採用されている部分型。

どちらも部分型の定義としてリスコフの置換原則を満たしており、どちらが正しいというものではない。

Python は動的型付け言語なので、部分型は型アノテーションに関する型の代替性（これは静的型チェックツールによって確認される）の話題となる。

Python においても、静的型チェックツールは名前的部分型を認識できる。次のコードでは、`Pigeon` クラスは抽象基底クラス `Bird` から継承している。`Pigeon` は `Bird` であるから代入可能である。

In [ ]:
from abc import ABC, abstractmethod


class Bird(ABC):
    @abstractmethod
    def fly(self) -> None: ...


class Pigeon(Bird):
    def fly(self) -> None:
        print("ハトが空を飛ぶ")

    def run(self) -> None:
        print("ハトが素早く走る")


def main(bird: Bird):
    bird.fly()


if __name__ == "__main__":
    main(Pigeon())  # 静的型チェックをパスする

ハトが空を飛ぶ


実のところ、このコードは継承を使わないように変更しても、静的型チェックツールはエラーを出力するもののプログラムそのものは問題なく動作する:

In [ ]:
class Bird:
    def fly(self) -> None: ...


class Pigeon:
    def fly(self) -> None:
        print("ハトが空を飛ぶ")

    def run(self) -> None:
        print("ハトが素早く走る")


def main(bird: Bird):
    bird.fly()


if __name__ == "__main__":
    main(Pigeon())  # type: ignore  ← このコメントを消すと静的型チェックツールはエラーを出力する

ハトが空を飛ぶ


これは、ダックタイピングの一例である。静的型チェックツールは動的に型の代替性を判断できないので、最初のコードでは継承を使ったのであるが、この手法は型の代替性をクリアしている場面でわざわざ継承を使っていることになる。継承を使うことによって新たな問題を抱える可能性があることを考慮しなければならない。

Protocol
--------

`typing` モジュールは、抽象基底クラス `Protocol` を提供している。このクラスは、静的型チェックツールが構造的部分型を認識する際に使うものである。`typing.Protocol` を使うコードでは、構造的部分型は静的にチェックできるダックタイピングと捉えることができる。

基底型は、`typing.Protocol` を継承するクラスの形で定義する。このクラスはインスタンス化しようとすると `TypeError`を送出する。このクラスの利用は静的型チェックに限られるので、抽象メソッドのようにメソッドのシグネチャだけを宣言し、処理を書かない。

次のコードでは、継承を使って `typing.Protocol` 型の `Bird` を定義し、`main()` 関数の `Bird` 型の引数として `Pigeon` インスタンスを渡している。`typing.Protocol` に対応する静的型チェックツールは、`Pigeon` が `Bird` の構造的部分型であると認識し、エラーを出力しない。

In [ ]:
from typing import Protocol


class Bird(Protocol):
    def fly(self) -> None: ...


class Pigeon:
    def fly(self) -> None:
        print("ハトが空を飛ぶ")

    def run(self) -> None:
        print("ハトが素早く走る")


def main(bird: Bird):
    bird.fly()


if __name__ == "__main__":
    main(Pigeon())  # 静的型チェックをパスする

ハトが空を飛ぶ


このように、継承を使いたくないが、静的型チェックの恩恵は受けたい場合、`typing.Protocol` 型を使うとよい。

以降では、`typing.Protocol` 型を定義するクラスを**プロトコルクラス**と呼ぶ。`collections.abc` モジュールが提供する抽象基底クラスは、抽象基底クラスでありながら、プロトコルクラスとしても使うことができる。

In [ ]:
from collections.abc import Callable


def func(x: int, checker: Callable[[int], bool]) -> int:
    return x if checker(x) else x * 3


class MyChecker:
    def __call__(self, x: int) -> bool:
        return (x % 5) == 0


if __name__ == "__main__":
    my_checker = MyChecker()
    assert func(20, my_checker) == 20  # MyChecker は Callable を継承していないが、構造的部分型である

プロトコルクラスのインスタンス変数またはクラス変数は、クラス本体内で明示的に宣言する必要がある:

``` python
class User(Protocol):
    name: str
```

プロトコルクラスはジェネリックにもできる。Python 3.12 以降では

``` python
class GenProto[T](Protocol):
    def meth(self) -> T:
        ...
```

Python 3.11 以前ならば

``` python
T = TypeVar("T")

class GenProto(Protocol[T]):
    def meth(self) -> T:
        ...
```

Python 3.13 で、`typing` モジュールに次の 2 つの関数が追加された。

``` python
typing.get_protocol_members(tp)
```

指定したプロトコルクラスに定義されているメンバー（メソッドや属性）の名前を `frozenset` として返す。

``` python
typing.is_protocol(tp)
```

指定したクラスがプロトコルクラスかどうかを判定する。

アダプターパターン
------------------

**アダプターパターン**（adapter pattern）は、デザインパターンの 1 つで、既存のクラスに対して修正を加えることなく、そのクラスと関係のないインターフェースを実装するための変換用クラスを用意する設計パターンである。以下の 3 つのメンバーで構成される。

  * **Adaptee**: 既存のクラス。第三者から提供されるモジュール内に存在しているなどの事情によりこれは修正できない
  * **Target**: Adaptee に持たせたいインターフェース
  * **Adapter**: Adaptee インスタンスを持ち、Target インターフェースを実装するメソッドの中で Adaptee インスタンスのメソッドを呼び出す

Adapter は、Target インターフェースを持つように Adaptee を変換するクラスであり、Adaptee の代わりに使用される。Target は、もはや Adapter の構造を示すだけとなるので、Python ではプロトコルクラスとして作成する。

![](https://www.plantuml.com/plantuml/png/PP0nJm9148Nx_HLJQd7ORyW5xEty0-Fj75ootk3iN67KGcoqC9PQQsK1j9gG-6CEQFaBAjxY4QgtoNlvTjxh1KNHwjoRebII73E2KvG1gVaJi725Pq45CxqHkgWA8NT2EcHtS6WoZo50t-58IKoZO04tBmBRUDJozY0bAu4KWw42vn5cxLuE893FEmsxm_xVejB1BsRd86gHBQoUNvUJb_fsMezdzTrxFVxiPknPhF0b-cDVKRgUto--fgl9s-BZSVaqMyyVkaURJ6hwsqBjRsEfn5EkoNKQfulff1ebl9sdHwNxMVeR)

アダプターパターンは、既存のクラスが修正できない場合にインタフェースをそろえることに使用される。

例えば、以下のコードでは、`Cat` クラスは既存のクラスであり修正できないものとする。いま `Cat` クラスを利用したい開発者がいて、その開発者は `chatter` というメソッドで `Cat` の鳴き声を印字したいとする。この場合、`CatAdapter` という Adapter を作成することで、既存クラス（`Cat` クラス）を修正することなく、異なるインタフェースを持たせることができる。

In [ ]:
from typing import Protocol


class Cat:
    def meow(self) -> None:
        print("Cat: meow!")


class Chatter(Protocol):
    def chatter(self) -> None: ...


class CatAdapter:
    def __init__(self) -> None:
        self._cat = Cat()

    def chatter(self) -> None:
        self._cat.meow()


class Human:
    def chatter(self):
        print("Human: I'm home.")


def main(objs: list[Chatter]):
    for obj in objs:
        obj.chatter()


if __name__ == "__main__":
    objs = [Human(), CatAdapter()]
    main(objs)

Human: I'm home.
Cat: meow!


このように、既存クラスを修正することなく、異なるインタフェースを持たせるということが、アダプターパターンの役割である。

このアダプターの例を見ると、クラスの合成を使っただけである。アダプターパターンは、クラスの合成を使う最も基本的なパターンに名前を付けただけのデザインパターンと言える。重要なのは、「インターフェースを実装するラッパーで既存のコードをラップする」という設計である。

インターフェース分離の原則
--------------------------

**インターフェース分離の原則**（Interface Segregation Principle; ISP）とは、継承先で使わないメソッドがないようにインターフェースを分けたほうがよいという設計原則である。

同様の考え方は Mixin でもしていた。アダプターパターンにおいては、対象とするインターフェースを小分けにし、 Adapter は必要最小限のインターフェースを実装するようにする。

プロキシパターン
----------------

**プロキシパターン**は、デザインパターンの 1 つで、あるクラスを利用するときに、それを直接利用するのではなく、代理（Proxy）として機能するクラスを利用する設計パターンである。以下の 3 つのメンバーで構成される。

  * **RealSubject**: 本質的な処理を担当するクラス
  * **Proxy**: RealSubject インスタンスを保持するラッパーであり、本質的な処理の部分は RealSubject インスタンスのメソッドに委譲する
  * **Subject**: RealSubject と Proxy の共通インターフェース（Python ではプロトコルクラス）

ラッパーを使う点で、プロキシパターンとアダプターパターンは類似するが、Adapter はラップされたオブジェクトに対しては異なるインターフェースを提供するのに対し、 Proxy は同じインターフェースを提供する。

![](https://www.plantuml.com/plantuml/png/LP2n3S8m44LxJt4b5FO0AQACW622o-KWHiP6zaK20Lkm0nKDEo4n1X0IuNJMVr_FNrSbmIZTpYdqFWYA3Hx8MVQYh1UEQoI6LLTlcKIHmvHW4SFXEBoNZ6wahMqOSesDv5WD60euOPqa8ia5ce3qBStcAbD0aOF-dsdqMmR6HBjfnPXod3QKTRVNvt-OgpEKzNKlqhg4wOpdxVgwFrJ5llaSvmq0)

client 自身は、インターフェースだけを知り、それが RealSubject によって処理されるのか、Proxy によって代理されて処理されるのかを知らないことに注意する。どちらによって処理されるかは、実行時に決められる。

次のコードでは、メインのクラスは `RealSubject` である。`RealSubject` オブジェクトと直接やり取りする代わりに `Proxy` オブジェクトとやり取りすることができ、この場合はログ出力も行われる。`client` 関数は Subject インターフェースを実行するだけであり、どちらのオブジェクトが使われるかは実行時の引数によって決まる。

In [ ]:
from typing import Protocol


class Subject(Protocol):
    def doaction(self, user: str) -> None: ...


class RealSubject:
    def doaction(self, user: str) -> None:
        print(f"I am doing the job for {user}")


class Proxy:
    def __init__(self) -> None:
        self._realsubject = RealSubject()

    def doaction(self, user: str) -> None:
        print(f"[log] Doing the job for {user} is requested.")

        if user == "admin":
            self._realsubject.doaction(user)
        else:
            print("[log] I can do the job just for `admins`.")


def client(job_doer: Subject, user: str) -> None:
    job_doer.doaction(user)


def main():
    proxy = Proxy()
    client(proxy, "admin")
    client(proxy, "anonymous")
    print("-" * 48)
    real_subject = RealSubject()
    client(real_subject, "admin")
    client(real_subject, "anonymous")


if __name__ == "__main__":
    main()

[log] Doing the job for admin is requested.
I am doing the job for admin
[log] Doing the job for anonymous is requested.
[log] I can do the job just for `admins`.
------------------------------------------------
I am doing the job for admin
I am doing the job for anonymous


単一責任の原則
--------------

**単一責任の原則**（Single Responsibility Principle; SRP）とは、モジュール、クラスまたは関数は、単一の機能について責任を持ち、その機能をカプセル化（1 つのまとまりにすること）するべきであるという設計原則である。

あるコードがどれだけそのクラスの責任分担に集中しているかを示す度合いを**凝集度**（cohesion）という。凝集度が高い場合、コードの可読性が高くなり、テストが容易になるとされる。クラスが複数の機能について責任を持つ場合、ある機能に関するコードは、別の機能にとっては関係のないコードであり、凝集度が低下する。よって、単一責任の原則は、クラスの凝集度に関する原則といえる。

普遍的な処理に、ログ出力などシステム固有の処理（**ビジネスロジック**と呼ばれる）が入り込むと、メソッドはさまざまな付随的処理が組み合わさったものになる。プロキシパターンを使うことで、本質ではない処理をメソッドから分離することができ、クラスの凝集度を高めることになる。

ファクトリメソッドパターン
--------------------------

**ファクトリメソッドパターン**（factory method pattern）は、デザインパターンの 1 つで、テンプレートメソッドパターンをオブジェクト生成の場面に適応させた設計パターンである。すなわち、オブジェクト生成のインターフェースだけを決めておいて、具体的な処理（実際にどの型のオブジェクトを生成するのか）については、別のクラスで実装しなければならない。Python では継承を使わず、プロトコルクラスを使うことができる。ファクトリメソッドパターンは、以下の 4 つのメンバーで構成される。

  * **Product**: 生成されるすべてのオブジェクトに共通なインターフェース（Python ではプロトコルクラス）
  * **ConcreteProduct**: Product インターフェースの種々の異なる実装
  * **Creator**: 新しい Product オブジェクトを生成するためのインターフェース──そのメソッドはファクトリメソッドと呼ばれる
  * **ConcreteCreator**: Creator インターフェースの種々の異なる実装（ConcreteProduct オブジェクトを生成する）

Creator クラスでオブジェクトを作成するためのインターフェースが決まっているが、作成されるオブジェクトの型を個々の ConcreteCreator で変更することができる。

![](https://www.plantuml.com/plantuml/png/ZP113eCW403ll89ZUv0764CKFz0VD1ItrKIbMPP3q_Rl9OqOYyRqH38iCs5BHnh93pthHWAyQWEyHj1aaKqdp_bJNnoXDlJcOIAmnta0QcrpE8RR4zh66reO8Z6zTexNTZG819ElOYdW-POsJasoAjesSHfHB0L_HLGQYNXUXX3OtLeIGiPV1ajHU0TIhZ-M-NlsalD64ssyZwRL-sKjVVlD1g9fewnujfyE0_jjjTDMl-qIniOF_GS0)

製品番号（P/N）で区別される製品オブジェクトを生成する場面を想定すると、Pythonでの最低限の実装は以下のようになる。

In [ ]:
from typing import Protocol
from dataclasses import dataclass
import time


class Product(Protocol):
    product_number: str


class Creator(Protocol):
    def factory_method(self) -> Product: ...


@dataclass(frozen=True, slots=True)
class ConcreteProduct:
    serial_number: str
    product_number = "SOMEPRODUCT001"


class ConcreteCreator:
    def factory_method(self) -> Product:
        return ConcreteProduct(self.get_serial_number())

    def get_serial_number(self) -> str:
        return time.strftime(r"SP001%Y%m%d%H%M%S")

依存性の注入
------------

ファクトリメソッドパターンのメンバーを使用してオブジェクトを生成するメインのクラスを考える。メインのクラスを `Logic` とし、オブジェクト生成を行って処理をするメソッドを `do()` メソッドとする。オブジェクト生成を行うには ConcreteCreator インスタンスが必要なので、素直に `do()` メソッド内で（条件により）クラスを決めて ConcreteCreator インスタンスを入手するとすれば、次のようなコードを書く:

``` python
class Logic:
    def do(self):
        if CONDITION:
            creator = ConcreteCreatorA()
        else:
            creator = ConcreteCreatorB()
        self.product = creator.factory_method()
        ...  # 処理を書く
```

このように直接に ConcreteCreator のインスタンス化を書く場合、ConcreteCreator のクラスを増減する変更が `Logic` クラスにも影響する。 `Logic` が ConcreteCreator 側の変更による影響を受けないようにするには、 ConcreteCreator のインスタンス化を外部に追い出し、 `Logic` はただ Creator オブジェクトを受け取るだけにすればよい。

このように、あるクラス内で使用するオブジェクトをクラス外で生成することを**依存性の注入**（dependency injection; DI）と呼ぶ。通常であればクラスに依存しなければそのインスタンスを入手することはできないが、クラスの外でインスタンスを作成し、それを注入（インスタンスを貰うこと）することで、クラスに依存せずそのインスタンスのみを使用することができる。

Pythonでの最低限の実装は以下のようになる。

In [ ]:
class Logic:
    def __init__(self, creator: Creator):
        self.creator = creator

    def do(self):
        self.product = self.creator.factory_method()
        print(f"{self.product=}")
        print(f"{self.product.product_number=}")
        ...  # 処理を書く


def main():
    logic = Logic(ConcreteCreator())
    logic.do()


if __name__ == "__main__":
    main()

self.product=ConcreteProduct(serial_number='SP00120241018142453')
self.product.product_number='SOMEPRODUCT001'


DI により、`Logic` は Creator の部分型のオブジェクトを受け取ることは知っているが、それがどの ConcreteCreator インスタンスであるのかまでは知らない。こうして、`Logic` と ConcreteCreator の間の結合度を下げることができる。

依存性逆転の原則
----------------

**依存性逆転の原則**（dependency inversion principle; DIP）とは、次のような設計原則である。

  1. 上位モジュールは、下位モジュールに依存してはならない。どちらも抽象に依存するべきである。
  2. 抽象は詳細に依存してはならない。詳細が抽象に依存するべきである。

ここでいうモジュールとは、プログラムを構成する要素という意味であり、Python においてライブラリとしてのモジュールはもちろん、クラスや関数も該当する。上位と下位とは、モジュールやクラスの階層関係を表す。上位のモジュールやクラスは、下位のモジュールやクラスによって提供される機能やデータを利用するものである。例えば、アプリケーションのロジックを担当するクラスは上位のモジュールであり、外部リソースへのアクセスなどの詳細実装を担うクラスは下位のモジュールである。クラス A の定義中にクラス B を書くとき、クラス A はクラス B に「依存する」という。これを依存方向 A → B と書けば、依存性逆転の原則は上位から下位への依存方向があってはならないことを言っている。これにより、クラス間を疎結合に保つことができる。

ファクトリメソッドパターンと依存性の注入の例では、次のようになる。

``` text
　（上位モジュール） Logic クラス、 Product クラス、 Creator クラス  
　────────────────────────────────────────────────  
　（下位モジュール） main() 関数、 ConcreteProduct クラス、 ConcreteCreator クラス

　（依存方向） main()　→　Logic　→　Creator
　　　　　　　　　　　  ↳　ConcreteCreator　→　Creator
　　　　　　　　　　　　　　　　　　　　　　 ↳　ConcreteProduct　→　Product
```

上下をまたぐ依存方向は `main()`→`Logic` と `ConcreteCreator`→`Creator`、`ConcreteProduct`→`Product` だけであり、すべて下位から上位への方向である。つまり、ファクトリメソッドパターンと依存性の注入は「上位モジュールは、下位モジュールに依存してはならない」という原則に従っているのである。

ファクトリパターン
------------------

ファクトリメソッドパターンでは、 ConcreteProduct のクラスを増やすと、それに応じて ConcreteCreator のクラスも増やすことになり、コードが複雑になる。そこで、 ConcreteCreator のクラスたちを 1 つの `Factory` クラスにまとめてしまい、 ConcreteProduct の各クラスには分岐で対応するようにすると、次のようなコードを書ける。

In [ ]:
from typing import Protocol
from dataclasses import dataclass
import time


class Product(Protocol):
    product_number: str


@dataclass(frozen=True, slots=True)
class ConcreteProduct:
    serial_number: str
    product_number = "SOMEPRODUCT001"


class Factory:
    def create(self, product_number: str) -> Product:
        if product_number == "SOMEPRODUCT001":
            return ConcreteProduct(self.get_serial_number())
        #
        # <--- Product の種類を増やす場合はここに分岐 elif を加える
        #
        else:
            raise ValueError(f"product number {product_number} not found")

    def get_serial_number(self) -> str:
        return time.strftime(r"SP001%Y%m%d%H%M%S")


class Logic:
    def do(self):
        factory = Factory()
        self.product = factory.create("SOMEPRODUCT001")
        print(f"{self.product=}")
        print(f"{self.product.product_number=}")
        ...  # 処理を書く


def main():
    logic = Logic()
    logic.do()


if __name__ == "__main__":
    main()

self.product=ConcreteProduct(serial_number='SP00120241018142453')
self.product.product_number='SOMEPRODUCT001'


このように、`Factory` クラスの中で条件分岐などを使ってオブジェクトを生成するクラスを使い分ける設計パターンは**ファクトリパターン**（factory pattern）と呼ばれる。

ファクトリパターンでは、上位の `Factory` クラスが下位の ConcreteProduct のクラスに強く依存し、ConcreteProduct 側に変更があれば直ちに `Factory` クラスの修正が必要となってしまう。それでも、ファクトリメソッドパターンよりはコードが簡潔になる。KISS の原則はこの場面でも当てはまるから、依存性逆転の原則のメリットをあまり活かせない場面（下位モジュールの変更や交換が頻繁に起こるものではない場合など）では、ファクトリパターンでよいように思う。

ストラテジーパターン
--------------------

**ストラテジーパターン**（strategy pattern）は、デザインパターンの 1 つで、同じインターフェースを実装する交換可能なアルゴリズムをいくつか定義して、プログラム実行時に外部からアルゴリズムを選択できるようにする設計パターンである。以下の 3 つのメンバーで構成される。

  * **Strategy**: アルゴリズムが実装する共通のインターフェース（Python ではプロトコルクラス）
  * **ConcreteStrategy**: アルゴリズム（Strategy インターフェースを実装するクラス）
  * **Context**: ConcreteStrategy インスタンスを外部から注入され、そのメソッドを呼び出すことで一部の処理を委譲する

ストラテジーパターンは、依存性の注入（DI）を使用するデザインパターンである。 Context 自身はどのアルゴリズムが選択されるかを知らず、 ConcreteStrategy のクラスの追加・削除に影響されない。

![](https://www.plantuml.com/plantuml/png/SoWkIImgAStDuKhEIImkLd3Epoj9hIZXWj8ALWh59KM99QdfbK1cC9J9bPTVaggGavfMef2V1jDYPLKCarPuic_kqxKpdZKiVzouxjdc5KzRbxxVq-7Y-NvF9rTgNee2I3jN9b1NKLfYSYh2s624LTM9II3ZaajgymjIWQAP-NaQcbm8HoONfPQamjGxSc6808qWGgCJ5uO8caPuGEWMPQPdbEZQAI0vqiQcj3ZLpWwBxRgb1Rer2BMOrc2eS006330Q0000)

以下は、データの保存先として複数のクラウドストレージを利用する場合に、それぞれのストレージクライアントを独立したクラスとして実装した感じのコードである。

In [ ]:
from typing import Protocol
import io


class Storage(Protocol):
    def get(self, uri: str) -> io.BytesIO: ...
    def put(self, data: bytes, uri: str) -> None: ...


class StorageA:
    def get(self, uri: str):
        s = "Data from {} in StorageA".format(uri)
        return io.BytesIO(s.encode(encoding="utf-8"))

    def put(self, data: bytes, uri: str):
        pass


class StorageB:
    def get(self, uri: str):
        s = "Data from {} in StorageB".format(uri)
        return io.BytesIO(s.encode(encoding="utf-8"))

    def put(self, data: bytes, uri: str):
        pass


class Context:
    def __init__(self, input_storage: Storage, output_storage: Storage, input_path: str, output_path: str):
        self.input_storage = input_storage
        self.output_storage = output_storage
        self.input_path = input_path
        self.output_path = output_path

    def run(self):
        data = self.get_data()
        print(data)
        ...  # データの加工
        self.save_data(data)

    def get_data(self):
        io = self.input_storage.get(self.input_path)
        data = io.read()
        return data.decode(encoding="utf-8")

    def save_data(self, data):
        self.output_storage.put(data, self.output_path)


if __name__ == "__main__":
    context = Context(StorageA(), StorageA(), "sa://mybucket/olditem", "sa://mybucket/newitem")
    context.run()

Data from sa://mybucket/olditem in StorageA


Python では、関数はオブジェクトであり、アルゴリズムを関数として定義して他のオブジェクトに渡すことができるので、ストラテジーパターンを明示的に定義するまでもない場合も多い。

``` python
from collections.abc import Callable


def concrete_strategy_a(data: bytes) -> bytes:
    ...


def concrete_strategy_b(data: bytes) -> bytes:
    ...


class Context:
    def __init__(self, strategy: Callable[[bytes], bytes], output_path):
        self.strategy = strategy
        self.output_path = output_path

    def run(self):
        data = self.get_data()
        new = self.strategy(data)
        with open(self.output_path, "wb") as f:
            f.write(new)

    def get_data(self) -> bytes:
        ...
```

開放閉鎖原則
------------

**開放閉鎖原則**（Open/Closed Principle; OCP）とは、ソフトウェアの設計原則の一つで、「ソフトウェアの要素（クラス、モジュール、関数など）は拡張には開かれているが、修正には閉じているべきである」というものである。これは、新しい機能や変更が必要なとき、既存のコードを変更するのではなく、新しいコードを追加することでその要件を満たすことができるような設計を求めている。これにより、新しい機能や変更に関するテストだけを行って、既存のコードに対するテストを再実行する必要はないようにできる。

ストラテジーパターンに従うと、新しいアルゴリズムの追加は、継承を使わずに新しいクラスの追加によって容易に実現される。つまり、機能の拡張には開かれている。一方、アルゴリズムの修正はそのクラスの修正にとどまるから、修正には閉じている。こうして、ストラテジーパターンは開放閉鎖原則と調和を保つことができる。

SOLID 原則
----------

次の 5 つの原則の頭文字をとって SOLID と呼ばれる。

  1. 単一責任の原則（**S**ingle-responsibility principle）
  2. 開放閉鎖の原則（**O**pen/closed principle）
  3. リスコフの置換原則（**L**iskov substitution principle）
  4. インターフェース分離の原則（**I**nterface segregation principle）
  5. 依存性逆転の原則（**D**ependency inversion principle）
